In [329]:
import os
import re
import zipfile
import glob
import pandas as pd
import sys
import time
import datetime
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
from pandas_datareader import data


In [330]:
def get_share_price2(shareCode5):

    start = datetime.date(2022,2,1)
    end = datetime.date(2022,2,21)
    sharecd = "{0}.JP".format(shareCode5[0:4])
    csvname = './data/{0}.csv'.format(sharecd)

    if os.path.exists(csvname):
        df = pd.read_csv(csvname)
    else:
        time.sleep(5) 
        df = data.DataReader(sharecd,'stooq', start, end)
        df = df.rename(columns={'Close': 'close'})
        df.head()
        df.to_csv(csvname,encoding="shift-jis")
    
    return df

In [340]:
def get_share_price(shareCode5):
    sharecd = "{0}.T".format(shareCode5[0:4])
    csvname = './data/{0}.csv'.format(sharecd)
    if os.path.exists(csvname):
        df = pd.read_csv(csvname)
    else:
        my_share = share.Share(sharecd)
        symbol_data = None
        try:
            time.sleep(2)
            symbol_data = my_share.get_historical(
                share.PERIOD_TYPE_DAY, 10,
                share.FREQUENCY_TYPE_DAY, 1)
        except YahooFinanceError as e:
            print(e.message)

        df = pd.DataFrame(symbol_data)
        df.head()

        if df.size > 0:
            df['datetime'] = pd.to_datetime(df.timestamp, unit="ms")
            #日本時間へ変換
            df["datetime_JST"] = df["datetime"] + datetime.timedelta(hours=9)
            df.head()
            df.to_csv(csvname, encoding="shift-jis")

    return df

In [341]:
def unzip_file(zip_dir,xbrl_file_expressions):
    zip_files = glob.glob(os.path.join(zip_dir, '*.zip'))

    number_of_zip_lists = len(zip_files)
    print("number_of_zip_lists：", number_of_zip_lists)

    xbrl_files = glob.glob(xbrl_file_expressions)
    return xbrl_files


In [342]:
def make_edinet_info_list(edinetcodedlinfo_filepath):
    edinet_info = pd.read_csv(edinetcodedlinfo_filepath, skiprows=1,
                                 encoding='cp932')
    edinet_info = edinet_info[["ＥＤＩＮＥＴコード", "提出者業種"]]
    edinet_info_list = edinet_info.values.tolist()
    return edinet_info_list

In [343]:
import sys
sys.path.append(r'./Arelle')
from Arelle.arelle import Cntlr

def make_edinet_company_info_list(xbrl_files,edinet_info_list):
    edinet_company_info_list = []

    for index, xbrl_file in enumerate(xbrl_files):
        edinet_code = ""  # EDINETCODE
        security_code = ""  # 証券コード
        filer_name_jp = ""  # 企業名
        industry_code = ""  # 業種
        operating_income = ""  # 営業利益
        current_assets = ""  # 流動資産
        current_liabilities = ""  #流動負債
        investments_and_other_assets = ""  #投資その他の資産
        noncurrent_liabilities = ""  #固定負債
        total_of_issued_share = ""  # 発行済株式数
        
        company_info_list = []  # 企業情報

        ctrl = Cntlr.Cntlr(logFileName='logToPrint')
        model_xbrl = ctrl.modelManager.load(xbrl_file)

        print(xbrl_file, ":", index + 1, "/", len(xbrl_files))

        for fact in model_xbrl.facts:
        
            #print(fact.concept.label)
            
            if fact.concept.qname.localName == 'EDINETCodeDEI':
                print("EDINETコード", fact.value)
                edinet_code = fact.value

                for code_name in edinet_info_list:
                    if code_name[0] == edinet_code:
                        print("業種",code_name[1])
                        industry_code = code_name[1]
                        break
            elif fact.concept.qname.localName == 'SecurityCodeDEI':
                print("証券コード", fact.value)
                security_code = fact.value

            elif fact.concept.qname.localName == 'FilerNameInJapaneseDEI':
                print("企業名", fact.value)
                filer_name_jp = fact.value

            elif fact.concept.qname.localName in ['TotalNumberOfIssuedSharesSummaryOfBusinessResults', 'NumberOfIssuedSharesAsOfFiscalYearEndIssuedSharesTotalNumberOfSharesEtc']:
                print("発行済株式数", fact.value)
                total_of_issued_share = fact.value

            elif fact.concept.qname.localName in ['OperatingIncome', 'OrdinaryIncomeLossSummaryOfBusinessResults']:
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearDuration', 'CurrentYTDDuration', 'CurrentQuarterDuration', 'InterimDuration' ]:
                        print("営業利益(連結)", fact.value)
                        operating_income = fact.value
                    elif ctx.id in [ 'CurrentYTDDuration_NonConsolidatedMember', 'CurrentQuarterDuration_NonConsolidatedMember', 'CurrentYearDuration_NonConsolidatedMember', 'InterimDuration_NonConsolidatedMember' ]:
                        if operating_income == "":
                            print("営業利益(単体)", fact.value)
                            operating_income = fact.value

            elif fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None) == '流動資産':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("流動資産(連結)", fact.value)
                        current_assets = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if current_assets == "":
                            print("流動資産(単体)", fact.value)
                            current_assets = fact.value

            elif fact.concept.label(preferredLabel=None, lang='ja', linkroleHint=None) == '流動負債':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("流動負債(連結)", fact.value)
                        current_liabilities = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if current_liabilities == "":
                            print("流動負債(単体)", fact.value)
                            current_liabilities = fact.value

            elif fact.concept.qname.localName == 'InvestmentsAndOtherAssets':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("投資その他の資産(連結)", fact.value)
                        investments_and_other_assets = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if investments_and_other_assets == "":
                            print("投資その他の資産(単体)", fact.value)
                            investments_and_other_assets = fact.value

            elif fact.concept.qname.localName == 'NoncurrentLiabilities':
                ctx = fact.context
                if ctx is not None:
                    if ctx.id in [ 'CurrentYearInstant', 'CurrentQuarterInstant', 'InterimInstant' ]:
                        print("固定負債(連結)", fact.value)
                        noncurrent_liabilities = fact.value
                    elif ctx.id in [ 'CurrentYearInstant_NonConsolidatedMember', 'CurrentQuarterInstant_NonConsolidatedMember', 'InterimInstant_NonConsolidatedMember' ]:
                        if noncurrent_liabilities == "":
                            print("固定負債(単体)", fact.value)
                            noncurrent_liabilities = fact.value


        print("")
        
        if security_code != "" and operating_income != "":
            price = get_share_price(security_code)
            print(price)
            if price.size > 0:
                close_price = float(price['close'][0])
            else:
                close_price = 0.0
        else:
            close_price = 0.0

        if operating_income == "":
            operating_income = 0.0
        else:
            operating_income = float(operating_income)

        if current_assets == "":
            current_assets = 0.0
        else:
            current_assets = float(current_assets)

        if current_liabilities == "":
            current_liabilities = 0.0
        else:
            current_liabilities = float(current_liabilities)

        if investments_and_other_assets == "":
            investments_and_other_assets = 0.0
        else:
            investments_and_other_assets = float(investments_and_other_assets)
        
        if noncurrent_liabilities == "":
            noncurrent_liabilities = 0.0
        else:
            noncurrent_liabilities = float(noncurrent_liabilities)

        if total_of_issued_share == "":
            total_of_issued_share = 1
        else:
            total_of_issued_share = float(total_of_issued_share)

        # 事業価値
        business_value = operating_income * 10.0
        
        # 財産価値
        property_value = current_assets - (current_liabilities * 1.2) + investments_and_other_assets
        
        # 株主価値
        shareholder_value = business_value + property_value - noncurrent_liabilities
        
        # 1株の価値
        share_unit_value = shareholder_value / total_of_issued_share
        
        # 安全率
        if close_price == 0:
            safety_ratio = 0
        else:
            safety_ratio = share_unit_value / close_price
        
        company_info_list.append(edinet_code)
        company_info_list.append(security_code)
        company_info_list.append(filer_name_jp)
        company_info_list.append(industry_code)
        company_info_list.append(operating_income)
        company_info_list.append(current_assets)
        company_info_list.append(current_liabilities)
        company_info_list.append(investments_and_other_assets)
        company_info_list.append(noncurrent_liabilities)
        company_info_list.append(business_value)
        company_info_list.append(property_value)
        company_info_list.append(shareholder_value)
        company_info_list.append(total_of_issued_share)
        company_info_list.append(share_unit_value)
        company_info_list.append(close_price)
        company_info_list.append(safety_ratio)
        
        print(company_info_list)
        print("")

        edinet_company_info_list.append(company_info_list)

    return pd.DataFrame(data=edinet_company_info_list, columns=['EDINETコード','証券コード','企業名','業種','営業利益','流動資産','流動負債','投資その他の資産','固定負債','事業価値','財産価値','株主価値','発行株数','1株価値','株価','安全率'])


In [346]:
def main():
    edinetcodedlinfo_filepath = r'./EdinetcodeDlInfo.csv'
    edinet_info_list = make_edinet_info_list(edinetcodedlinfo_filepath)

    zip_dir = r'./data/'
    xbrl_file_expressions = r'./data/XBRL/PublicDoc/*.xbrl'
    xbrl_files = unzip_file(zip_dir,xbrl_file_expressions)

    edinet_company_info_list = make_edinet_company_info_list(xbrl_files, edinet_info_list)
    print(edinet_company_info_list)

    edinet_company_info_list.to_csv('./share_value_result.csv',encoding="utf-8")


In [ ]:
if __name__ == "__main__":
    main()